## Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import sklearn.metrics
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib.font_manager import FontProperties
prop = FontProperties()
plt.style.use('seaborn-dark-palette')
prop.set_file('/content/drive/My Drive/Bengali Grapheme/data/kalpurush.ttf')
%matplotlib inline

In [3]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from IPython.display import SVG
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Lambda, add
from tensorflow.keras.layers import concatenate

In [0]:
dataframe = pd.read_parquet('/content/drive/My Drive/Bengali Grapheme/data/train_labels.parquet')

In [0]:
X = dataframe.drop(['image_id','grapheme_root','vowel_diacritic','consonant_diacritic'],axis=1).values

In [0]:
y = dataframe[['grapheme_root','vowel_diacritic','consonant_diacritic']].values

In [0]:
del dataframe

In [0]:
X = X/255
histories = []

---

## Generators, Augmentation and Callbacks

### Generator

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

del X
del y

### Augmentation

In [0]:
def apply_augmentation(image):
  augmentation_pipeline = A.Compose(
        [
            A.OneOf(
                [
                    # apply one of transforms to 50% of images
                    A.RandomContrast(), # apply random contrast
                    A.RandomGamma(), # apply random gamma
                    A.RandomBrightness(), # apply random brightness
                ],
                p = 0.5
            ),
            A.OneOf(
                [
                    # apply one of transforms to 50% images
                    A.ElasticTransform(alpha = 120,sigma = 120 * 0.05,alpha_affine = 120 * 0.03),
                    A.GridDistortion(),
                    A.OpticalDistortion(distort_limit = 2,shift_limit = 0.5),
                    #A.ShiftScaleRotate()
                ],
                p = 0.5
            ),
        ],
        p = 0.5
    )
  images_aug = augmentation_pipeline(image = image)['image']
  return images_aug

### Callbacks

In [0]:
class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())

In [0]:
def get_callbacks(chkpoint_name, log_name, batch_size,epochs):

  checkpoint_cb = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Bengali Grapheme/checkpoints/'
  +chkpoint_name,save_best_only=True)
  
  early_stopping_cb = keras.callbacks.EarlyStopping(monitor='root_loss', patience=10, restore_best_weights=True)

  root_reduceLR_cb = keras.callbacks.ReduceLROnPlateau(monitor='root_loss', factor=0.2,
                                                  patience=5, min_lr=0.0001)
  vowel_reduceLR_cb = keras.callbacks.ReduceLROnPlateau(monitor='vowel_loss', factor=0.2,
                                                  patience=5, min_lr=0.0001)
  consonant_reduceLR_cb = keras.callbacks.ReduceLROnPlateau(monitor='consonant_loss', factor=0.2,
                                                  patience=5, min_lr=0.0001)
  
  root_logdir = '/content/drive/My Drive/Bengali Grapheme/logs'
    
  batch_size = batch_size
  epochs = epochs
  max_lr = 0.5
  base_lr = max_lr/10
  max_m = 0.98
  base_m = 0.85

  cyclical_momentum = True
  augment = True
  cycles = 2.35
    
  def get_run_logdir(name):
    import time
    run_id = time.strftime(name)
    return os.path.join(root_logdir,run_id)

  run_logdir = get_run_logdir(log_name+'-run_%Y_%m_%d-%H_%M_%S')

  tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

  iterations = round(X_train.shape[0]/batch_size*epochs)
  iterations = list(range(0,iterations+1))
  step_size = len(iterations)/(cycles)
  cyclicLR_cb =  CyclicLR(base_lr=base_lr,
                            max_lr=max_lr,
                            step_size=step_size,
                            max_m=max_m,
                            base_m=base_m,
                            cyclical_momentum=cyclical_momentum)
    
  return [checkpoint_cb, 
          root_reduceLR_cb, vowel_reduceLR_cb, consonant_reduceLR_cb,
          tensorboard_cb, early_stopping_cb]

---

## Models

### Define Inception, Residual, Inceptual, DenseNet and FractalNet Module

In [0]:
def residual_module(layer_in, n_filters):
  merge_input = layer_in
  if layer_in.shape[-1] != n_filters:
    merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
    merge_input = BatchNormalization()(merge_input)
    
  conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
  bn1 = BatchNormalization()(conv1)
  
  # conv2
  conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(bn1)
  bn2 = BatchNormalization()(conv2)
	
  # add filters, assumes filters/channels last
  layer_out = add([bn2, merge_input])
  # activation function
  layer_out = Activation('relu')(layer_out)
  return layer_out

In [0]:
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(conv3)
	# 5x5 conv
	conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu', kernel_initializer='he_normal')(conv5)
	# 3x3 max pooling
	pool = MaxPool2D((3,3), strides=(1,1), padding='same')(layer_in)
	pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

In [0]:
def inceptual_module(layer_in, n_filters, f1, f2_in, f2_out, f3_in, f3_out, f4_out):

  merge_input = layer_in
  if layer_in.shape[-1] != n_filters:
    merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
    merge_input = BatchNormalization()(merge_input)
    
  inception1 = inception_module(merge_input, f1, f2_in, f2_out, f3_in, f3_out, f4_out)
  bn1 = BatchNormalization()(inception1)
  
  # conv2
  conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(bn1)
  bn2 = BatchNormalization()(conv2)
	
  # add filters, assumes filters/channels last
  layer_out = add([bn2, merge_input])
  # activation function
  layer_out = Activation('relu')(layer_out)
  return layer_out

In [0]:
def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x

def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters

def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AvgPool2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters

In [0]:
def tensorflow_categorical(count, seed):
  assert count > 0
  arr = [1.] + [.0 for _ in range(count-1)]
  return tf.random.shuffle(arr, seed)

def rand_one_in_array(count, seed=42):
  return tensorflow_categorical(count=count, seed=seed)

In [0]:
class JoinLayer(keras.layers.Layer):
  def __init__(self, drop_p, is_global, global_path, force_path, **kwargs):
    #print "init"
    self.p = 1. - drop_p
    self.is_global = is_global
    self.global_path = global_path
    self.uses_learning_phase = True
    self.force_path = force_path
    super(JoinLayer, self).__init__(**kwargs)
  
  def build(self, input_shape):
    #print("build")
    self.average_shape = list(input_shape[0])[1:]
    
  def _random_arr(self, count, p):
    return K.random_binomial((count,), p=p)

  def _arr_with_one(self, count):
    return rand_one_in_array(count=count)

  def _gen_local_drops(self, count, p):
    # Create a local droppath with at least one path
    arr = self._random_arr(count, p)
    drops = K.switch(K.any(arr),arr,self._arr_with_one(count))
    return drops

  def _gen_global_path(self, count):
    return self.global_path[:count]

  def _drop_path(self, inputs):
    count = len(inputs)
    drops = K.switch(self.is_global,self._gen_global_path(count),self._gen_local_drops(count, self.p))
    
    ave = K.zeros(shape=self.average_shape)
    for i in range(0, count):
      ave = ave + (inputs[i] * drops[i])
    sum = K.sum(drops)
    # Check that the sum is not 0 (global droppath can make it 0) to avoid divByZero
    ave = K.switch(K.not_equal(sum, 0.),ave/sum,ave)
    
    return ave

  def _ave(self, inputs):
    ave = inputs[0]
    for input in inputs[1:]:
      ave = ave + input
    ave = ave/len(inputs)
    return ave

  def call(self, inputs, mask=None):
    #print("call")
    if self.force_path:
      output = self._drop_path(inputs)
    else:
      output = K.in_train_phase(self._drop_path(inputs), self._ave(inputs))
    return output

  def get_output_shape_for(self, input_shape):
    #print("get_output_shape_for", input_shape)
    return input_shape[0]
  
  def get_config(self):
    base_config = super().get_config()
    return {**base_config}

In [0]:
class JoinLayerGen:
  
  def __init__(self, width, global_p=0.5, deepest=False):
    self.global_p = global_p
    self.width = width
    self.switch_seed = np.random.randint(1, 10e6)
    self.path_seed = np.random.randint(1, 10e6)
    self.deepest = deepest
    if deepest:
      self.is_global = K.variable(1.)
      self.path_array = K.variable([1.] + [.0 for _ in range(width-1)])
    else:
      self.is_global = self._build_global_switch()
      self.path_array = self._build_global_path_arr()

  def _build_global_path_arr(self):
    # The path the block will take when using global droppath
    return rand_one_in_array(seed=self.path_seed, count=self.width)

  def _build_global_switch(self):
    # A randomly sampled tensor that will signal if the batch
    # should use global or local droppath
    return K.equal(K.random_binomial((), p=self.global_p, seed=self.switch_seed), 1.)

  def get_join_layer(self, drop_p):
    global_switch = self.is_global
    global_path = self.path_array
    return JoinLayer(drop_p=drop_p, is_global=global_switch, global_path=global_path, force_path=self.deepest)

In [0]:
def fractal_conv(filters, nb_row, nb_col, dropout=None):
  def f(prev):
    conv = prev
    conv = Conv2D(filters, kernel_size=(nb_row, nb_col), kernel_initializer='he_normal', padding='same')(conv)
    if dropout:e
      conv = Dropout(dropout)(conv)
    conv = BatchNormalization(axis=-1)(conv)
    conv = Activation('relu')(conv)
    return conv
  return f

def fractal_block(join_gen, c, filters, nb_col, nb_row, drop_p, dropout=None):
  def f(z):
    columns = [[z] for _ in range(c)]
    last_row = 2**(c-1) - 1
    for row in range(2**(c-1)):
      t_row = []
      for col in range(c):
        prop = 2**(col)
        # Add blocks
        if (row+1) % prop == 0:
          t_col = columns[col]
          t_col.append(fractal_conv(filters=filters,nb_col=nb_col,
                                    nb_row=nb_row,
                                    dropout=dropout)(t_col[-1]))
          t_row.append(col)
        # Merge (if needed)
      if len(t_row) > 1:
        merging = [columns[x][-1] for x in t_row]
        merged  = join_gen.get_join_layer(drop_p=drop_p)(merging)
        for i in t_row:
          columns[i].append(merged)
    return columns[0][-1]
  return f

In [0]:
def fractal_net(b, c, conv, drop_path, global_p=0.5, dropout=None, deepest=False):
  def f(z):
    output = z
    # Initialize a JoinLayerGen that will be used to derive the
    # JoinLayers that share the same global droppath
    join_gen = JoinLayerGen(width=c, global_p=global_p, deepest=deepest)
    for i in range(b):
      (filters, nb_col, nb_row) = conv[i]
      dropout_i = dropout[i] if dropout else None
      output = fractal_block(join_gen=join_gen,
                             c=c, filters=filters,
                             nb_col=nb_col,nb_row=nb_row,
                             drop_p=drop_path,
                             dropout=dropout_i)(output)
      output = MaxPool2D(pool_size=(2,2), strides=(2,2))(output)
    return output
  return f

### Alexnet version 1

In [0]:
def model_alexnet():
    
    input_ = Input(shape=(64,64,1))

    augmentation = Lambda(apply_augmentation, output_shape=(64,64,1))(input_)

    conv1 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(augmentation)
    maxpool1 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv1)
    bn1 = BatchNormalization()(maxpool1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu')(bn1)
    maxpool2 = MaxPool2D(pool_size=(3, 3), strides=2, padding='valid')(conv2)
    bn2 = BatchNormalization()(maxpool2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(bn2)
    drop1 = Dropout(0.5)(conv3)
    conv4 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop1)
    drop2 = Dropout(0.5)(conv4)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(drop2)
    
    flat = Flatten()(conv5)
    
    dense_root_1 = Dense(4096,activation='relu')(flat)
    dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    output_root = Dense(168,activation='softmax',name='root')(dense_root_3)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [0]:
keras.backend.clear_session()
model_alexnet = model_alexnet()
#keras.utils.plot_model(model_alexnet, '../results/alexnet.png', expand_nested=True, show_shapes=True);

In [0]:
model_alexnet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [0]:
history_alexnet = model_alexnet.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=30, callbacks=get_callbacks('alexnet.h5','alexnet.h5',32,30))

In [0]:
model_alexnet.save('/content/drive/My Drive/Bengali Grapheme/models/alexnet_v1.h5')

### SayanNet v1 - Residual

In [0]:
def model_sayannet():
  
  input_ = Input(shape=(64,64,1))

  augmentation = Lambda(apply_augmentation)(input_)

  residual1 = residual_module(augmentation, 64)
  residual2 = residual_module(residual1,128)

  flat = GlobalAveragePooling2D()(residual2)

  output_root = Dense(168,activation='softmax',name='root')(flat)

  output_vowel = Dense(11,activation='softmax',name='vowel')(flat)
  
  output_consonant = Dense(7,activation='softmax',name='consonant')(flat)
    
  model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])

  return model


In [0]:
#keras.backend.clear_session()
model_sayannet = model_sayannet()
keras.utils.plot_model(model_sayannet, '/content/drive/My Drive/Bengali Grapheme/results/sayannet_v1.png', expand_nested=True, show_shapes=True);

In [0]:
model_sayannet.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [0]:
history_sayannet = model_sayannet.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=30, callbacks=get_callbacks('sayannet.h5','sayannet.h5',32,30))

In [0]:
model_sayannet.save('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v1.h5')

### SayanNet v2 - Inception 

In [0]:
def model_sayannet_v2():
  input_ = Input(shape=(64,64,1))
  #augmentation = Lambda(apply_augmentation, output_shape=(64,64,1))(input_)

  inception1 = inception_module(input_, 64, 96, 128, 16, 32, 32)
  # add inception block 1
  inception2 = inception_module(inception1, 128, 128, 192, 32, 96, 64)

  flat = GlobalAveragePooling2D()(inception2)

  output_root = Dense(168,activation='softmax',name='root')(flat)

  output_vowel = Dense(11,activation='softmax',name='vowel')(flat)
  
  output_consonant = Dense(7,activation='softmax',name='consonant')(flat)
    
  model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])

  return model


In [0]:
#keras.backend.clear_session()
model_sayannet_v2 = model_sayannet_v2()
keras.utils.plot_model(model_sayannet_v2, '/content/drive/My Drive/Bengali Grapheme/results/sayannet_v2.png', expand_nested=True, show_shapes=True);

In [0]:
model_sayannet_v2.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [0]:
history_sayannet_v2 = model_sayannet_v2.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=30, callbacks=get_callbacks('sayannet_v2.h5','sayannet_v2.h5',32,30))

In [0]:
model_sayannet_v2.save('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v2.h5')

### SayanNet v3 - Inceptual (Inception + Residual) Net

In [0]:
def model_sayannet_v3():
  input_ = Input(shape=(64,64,1))
  #augmentation = Lambda(apply_augmentation, output_shape=(64,64,1))(input_)

  inception1 = inceptual_module(input_, 64, 64, 96, 128, 16, 32, 32)
  inception2 = inceptual_module(inception1, 128, 128, 128, 192, 32, 96, 64)

  flat = GlobalAveragePooling2D()(inception2)

  dense_root_1 = Dense(4096,activation='relu')(flat)
  dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
  dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
  output_root = Dense(168,activation='softmax',name='root')(dense_root_3)
    
  dense_vowel_1 = Dense(800,activation='relu')(flat)
  dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
  dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
  output_vowel = Dense(11,activation='softmax',name='vowel')(dense_vowel_3)
    
  dense_consonant_1 = Dense(800,activation='relu')(flat)
  dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
  dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
  output_consonant = Dense(7,activation='softmax',name='consonant')(dense_consonant_3)
    
  model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])

  return model

In [0]:
#keras.backend.clear_session()
model_sayannet_v3 = model_sayannet_v3()
keras.utils.plot_model(model_sayannet_v3, '/content/drive/My Drive/Bengali Grapheme/results/sayannet_v3.png', expand_nested=True, show_shapes=True);

In [0]:
model_sayannet_v3.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [0]:
history_sayannet_v3 = model_sayannet_v3.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=30, callbacks=get_callbacks('sayannet_v3.h5','sayannet_v3.h5',32,30))

In [0]:
model_sayannet_v3.save('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v3.h5')

### SayanNet v4 - Dense Net

In [0]:
def model_sayannet_v4(filters=24, growth_rate=12, dense_block_size=3, layers_in_block=4):

  dense_block_size = 3
  layers_in_block = 4

  growth_rate = 12
  
  input_ = Input(shape=(64,64,1))
  #augmentation = Lambda(apply_augmentation, output_shape=(64,64,1))(input_)

  conv1 = Conv2D(24, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_)

  bn1 = BatchNormalization()(conv1)
  act_x = Activation('relu')(bn1)

  dense_x = MaxPool2D((3, 3), strides=(2, 2), padding='same')(act_x)
  for block in range(dense_block_size - 1):
    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x, filters = transition_block(dense_x, filters)

  dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
  bn_l = BatchNormalization()(dense_x)
  act_l = Activation('relu')(bn_l)
  flat = GlobalAveragePooling2D()(act_l)
  
  output_root = Dense(168,activation='softmax',name='root')(flat)

  output_vowel = Dense(11,activation='softmax',name='vowel')(flat)
  
  output_consonant = Dense(7,activation='softmax',name='consonant')(flat)
    
  model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])

  return model

In [0]:
#keras.backend.clear_session()
model_sayannet_v4 = model_sayannet_v4()
keras.utils.plot_model(model_sayannet_v4, '/content/drive/My Drive/Bengali Grapheme/results/sayannet_v4.png', expand_nested=True, show_shapes=True);

In [0]:
model_sayannet_v4.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [25]:
history_sayannet_v4 = model_sayannet_v4.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=100, callbacks=get_callbacks('sayannet_v4_noAug.h5','sayannet_v4_noAug.h5',32,100))

Train on 180756 samples, validate on 20084 samples
Epoch 1/100
180756/180756 [==============================] - 99s 548us/sample - loss: 2.6851 - root_loss: 1.8676 - vowel_loss: 0.4508 - consonant_loss: 0.3666 - root_accuracy: 0.5148 - vowel_accuracy: 0.8502 - consonant_accuracy: 0.8765 - val_loss: 1.6695 - val_root_loss: 1.1073 - val_vowel_loss: 0.2413 - val_consonant_loss: 0.3208 - val_root_accuracy: 0.6922 - val_vowel_accuracy: 0.9244 - val_consonant_accuracy: 0.8997
Epoch 2/100
180756/180756 [==============================] - 90s 498us/sample - loss: 1.2708 - root_loss: 0.8168 - vowel_loss: 0.2372 - consonant_loss: 0.2168 - root_accuracy: 0.7663 - vowel_accuracy: 0.9249 - consonant_accuracy: 0.9302 - val_loss: 1.3396 - val_root_loss: 0.8585 - val_vowel_loss: 0.2592 - val_consonant_loss: 0.2217 - val_root_accuracy: 0.7723 - val_vowel_accuracy: 0.9191 - val_consonant_accuracy: 0.9335
Epoch 3/100
180756/180756 [==============================] - 90s 496us/sample - loss: 1.0439 - root_l

In [0]:
model_sayannet_v4.save('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v4_noAug.h5')

### SayanNet v5 - Fractal Net

In [0]:
def model_sayannet_v5():
  
  deepest=False
  dropout = [0., 0.1, 0.2, 0.3, 0.4]
  conv = [(64, 3, 3), (128, 3, 3), (256, 3, 3), (512, 3, 3), (512, 2, 2)]

  input_= Input(shape=(64, 64,1))
  #augmentation = Lambda(apply_augmentation, output_shape=(64,64,1))(input_)
  output = fractal_net(c=3, b=5, conv=conv,drop_path=0.15, 
                       dropout=dropout,deepest=deepest)(input_)
  
  flat = Flatten()(output)

  output_root = Dense(168,activation='softmax',name='root')(flat)

  output_vowel = Dense(11,activation='softmax',name='vowel')(flat)
  
  output_consonant = Dense(7,activation='softmax',name='consonant')(flat)

  model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])

  return model

In [0]:
#keras.backend.clear_session()
model_sayannet_v5 = model_sayannet_v5()
keras.utils.plot_model(model_sayannet_v5, '/content/drive/My Drive/Bengali Grapheme/results/sayannet_v5.png', expand_nested=True, show_shapes=True);

In [0]:
model_sayannet_v5.compile(loss = [keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy,
                              keras.losses.sparse_categorical_crossentropy],
              optimizer = 'Adam',
              metrics=['accuracy'])

In [77]:
history_sayannet_v5 = model_sayannet_v5.fit(X_train.reshape(-1,64,64,1),
                                    (y_train[:,0], y_train[:,1],y_train[:,2]), 
                                    validation_data = (X_valid.reshape(-1,64,64,1), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
                                    batch_size=32, epochs=30, callbacks=get_callbacks('sayannet_v5.h5','sayannet_v5.h5',32,30))

Train on 180756 samples, validate on 20084 samples
Epoch 1/30
    32/180756 [..............................] - ETA: 8:23WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ValueError: ignored

## Model Evaluation

In [0]:
model_sayannet_v3 = tf.keras.models.load_model('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v3.h5')

In [0]:
model_sayannet_v3.evaluate(X_valid.reshape(-1,64,64,1),  (y_valid[:,0], y_valid[:,1],y_valid[:,2]))

66278/66278 [==============================] - 80s 1ms/sample - loss: 1.3984 - root_loss: 0.9235 - vowel_loss: 0.2498 - consonant_loss: 0.2248 - root_accuracy: 0.7737 - vowel_accuracy: 0.9274 - consonant_accuracy: 0.9333


[1.3984383012731951,
 0.92349756,
 0.2497758,
 0.22484691,
 0.77368057,
 0.9274118,
 0.93325084]

In [0]:
model_sayannet_v1 = tf.keras.models.load_model('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v1.h5')

In [0]:
model_sayannet_v1.evaluate(X_valid.reshape(-1,64,64,1),  (y_valid[:,0], y_valid[:,1],y_valid[:,2]))

66278/66278 [==============================] - 28s 423us/sample - loss: 3.1799 - root_loss: 1.7218 - vowel_loss: 0.5916 - consonant_loss: 0.8665 - root_accuracy: 0.5346 - vowel_accuracy: 0.7960 - consonant_accuracy: 0.6696


[3.1799310623165953,
 1.7218243,
 0.5915747,
 0.8664525,
 0.5346269,
 0.7959806,
 0.6696189]

In [0]:
model_sayannet_v2 = tf.keras.models.load_model('/content/drive/My Drive/Bengali Grapheme/models/sayannet_v2.h5')

In [0]:
model_sayannet_v2.evaluate(X_valid.reshape(-1,64,64,1),  (y_valid[:,0], y_valid[:,1],y_valid[:,2]))

66278/66278 [==============================] - 53s 806us/sample - loss: 2.4278 - root_loss: 1.4809 - vowel_loss: 0.5181 - consonant_loss: 0.4289 - root_accuracy: 0.6048 - vowel_accuracy: 0.8217 - consonant_accuracy: 0.8531


[2.427800789434522,
 1.480861,
 0.5180764,
 0.42888618,
 0.604801,
 0.82173574,
 0.8531338]

In [8]:
model_sayannet_v4 = tf.keras.models.load_model('../models/sayannet_v4_noAug.h5')

In [0]:
model_sayannet_v4.evaluate(X_valid.reshape(-1,64,64,1),  (y_valid[:,0], y_valid[:,1],y_valid[:,2]))

66278/66278 [==============================] - 17s 255us/sample - loss: 0.7083 - root_loss: 0.4101 - vowel_loss: 0.1452 - consonant_loss: 0.1527 - root_accuracy: 0.8861 - vowel_accuracy: 0.9594 - consonant_accuracy: 0.9559


[0.7082605381701996,
 0.41010314,
 0.14515221,
 0.15274046,
 0.8860708,
 0.95939827,
 0.95589787]

In [0]:
preds = model_sayannet_v4.predict(X_valid.reshape(-1,64,64,1))

In [0]:
result_root = np.zeros(66278,dtype=np.uint32)
for i in range(preds[0].shape[0]):
  result_root[i] = np.argmax(preds[0][i])
y_true = y_valid[:,0]

In [0]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_true, result_root,normalize='true'))

[[0.97619048 0.         0.         ... 0.         0.         0.        ]
 [0.09433962 0.86792453 0.         ... 0.         0.         0.        ]
 [0.         0.         0.84821429 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.91292135 0.00280899 0.        ]
 [0.         0.         0.         ... 0.         0.93814433 0.        ]
 [0.         0.         0.         ... 0.         0.         0.91825095]]


In [0]:
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    # Only use the labels that appear in the data
    classes = np.array(classes)
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    fig, ax = plt.subplots()
    fig.set_size_inches(30, 30)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [0]:
plot_confusion_matrix(y_true,result_root,classes=np.array(range(168),dtype=str))

In [17]:
from ann_visualizer.visualize import ann_viz
from tensorflow.keras.models import model_from_json
import keras

Using TensorFlow backend.


In [20]:
def myprint(s):
    with open('modelsummary.txt','w+') as f:
        print(s, file=f)

model_sayannet_v4.summary(print_fn=myprint)

In [22]:
print(model_sayannet_v4.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 64, 64, 24)   216         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 64, 64, 24)   96          conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 64, 64, 24)   0           batch_normalization_16[0][0]     
____________________________________________________________________________________________

In [24]:
sample = open('samplefile.txt', 'w')
print(model_sayannet_v4.summary(), file = sample) 
sample.close() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 64, 64, 24)   216         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 64, 64, 24)   96          conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 64, 64, 24)   0           batch_normalization_16[0][0]     
____________________________________________________________________________________________